# Moledetection



In [3]:
# START FIND MODEL 
"""
Skin cancer lesion classification using the HAM10000 dataset
Autokeras to find the best model. 
Dataset link:
https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000
Data description: 
https://arxiv.org/ftp/arxiv/papers/1803/1803.10417.pdf
The 7 classes of skin cancer lesions included in this dataset are:
Melanocytic nevi (nv)
Melanoma (mel)
Benign keratosis-like lesions (bkl)
Basal cell carcinoma (bcc) 
Actinic keratoses (akiec)
Vascular lesions (vas)
Dermatofibroma (df)

labels=['nv','mel','bkl','bcc','akiec','vas','df']
"""


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image

np.random.seed(42)
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import LabelEncoder
import autokeras as ak

skin_df = pd.read_csv('../backup/data/HAM10000/HAM10000_metadata.csv')


SIZE=64

# label encoding to numeric values from text
le = LabelEncoder()
le.fit(skin_df['dx'])
LabelEncoder()
#print(list(le.classes_))
 
skin_df['label'] = le.transform(skin_df["dx"]) 
#print(skin_df.sample(10))


# Distribution of data into various classes 
from sklearn.utils import resample
print(skin_df['label'].value_counts())

#Balance data.
# Many ways to balance data... you can also try assigning weights during model.fit
#Separate each classes, resample, and combine back into single dataframe
df=[]
for i in range(7):
    df.append( skin_df[skin_df['label'] == i])

###############################################
from sklearn.utils import shuffle
# split 20% test 20% validation 60% train for every class
df_test=[]
df_validate=[]
df_train=[]
for i in range(7):
    # shuffle the dataframe
    df[i] = shuffle(df[i], random_state=42)
    #split df in 20% as final testset
    len_df=len(df[i])
    len_test=int(len_df * 0.2)
    len_validation=len_test
    #len_rest=len_df - (len_test + len_validation)
    df_test.append(df[i].head(len_test))
    df[i]=df[i].tail(len_df - len_test)
    df_validate.append(df[i].head(len_validation))
    df_train.append(df[i].tail(len_df -len_validation -len_test))
##########################################
print('@$$$$$$$$$',df_train[0].shape,df_validate[0].shape,df_test[0].shape)
#input()
##########------------------###################################################################
# resample train set classes to each 500 samples 
n_samples=500
df_train_balanced=[]
for i in range(7):
    df_train_balanced.append( resample(df_train[i], replace=True, n_samples=n_samples, random_state=42) )
print('@$$$-----$',df_train_balanced[0].shape,df_train[0].shape,df_validate[0].shape,df_test[0].shape)
#input()
print('#prepare train set')
# prepare train set
# #########-------------###########################
def prepare_set(df_set,skin_df = skin_df):
    print(type(df_set))
    
    #put all classes together
    df_set = pd.concat([df_set[0], df_set[1], df_set[2], df_set[3], df_set[4], df_set[5], df_set[6]])
    #read images using image ID from the CSV file
    image_path = {os.path.splitext(os.path.basename(x))[0]: x
                        for x in glob(os.path.join('../backup/data/HAM10000/HAM10000_images_part_?', '*.jpg'))}
    #Define the path and add as a new column
    df_set['path'] = skin_df['image_id'].map(image_path.get)
    #Use the path to read images.
    df_set['image'] = df_set['path'].map(lambda x: np.asarray(Image.open(x).resize((SIZE,SIZE))))
    #Convert dataframe column of images into numpy array
    X = np.asarray(df_set['image'].tolist())
    X = X/255. # Scale values to 0-1. You can also used standardscaler or other scaling methods.
    Y=df_set['label'] #Assign label values to Y
    Y_cat = to_categorical(Y, num_classes=7) #Convert to categorical -> multiclass classification 
    return X, Y_cat
    #########-------------------------######################################################
# prepare all sets to be used
X_train_balanced,Y_cat_train_balanced = prepare_set(df_train_balanced)
X_test,Y_cat_test = prepare_set(df_test)
X_validate,Y_cat_validate = prepare_set(df_validate)

print('@$$$--x---$',X_test.shape,X_train_balanced.shape,X_validate.shape,'@@', df_train_balanced[0].shape,df_train[0].shape,df_validate[0].shape,df_test[0].shape)
#input()

print('@@@@@@@',len(X_train_balanced),len(X_test),len(X_validate))
# for selecting the best model we will use the validation set and split this into a train/test
#
# start model selection
print('# start model selection')
x_train_auto, x_test_auto, y_train_auto, y_test_auto = train_test_split(X_validate, Y_cat_validate, test_size=0.5, random_state=42)

# Define classifier for autokeras. Here we check 25 different models, each model 25 epochs
# 
clf = ak.ImageClassifier(max_trials=25) #MaxTrials - max. number of keras models to try
clf.fit(x_train_auto, y_train_auto, epochs=25)

#Evaluate the classifier on test data
print('#Evaluate the classifier on test data ')

_, acc = clf.evaluate(x_test_auto, y_test_auto)
print("Accuracy = ", (acc * 100.0), "%")

# get the final best performing model
print('# get the final best performing model ')

model = clf.export_model()
print(model.summary())
print('#Save the model')
'#Save the model'
model.save('../pybin/models/activemodeltest')



5    6705
4    1113
2    1099
1     514
0     327
6     142
3     115
Name: label, dtype: int64
@$$$$$$$$$ (197, 8) (65, 8) (65, 8)
@$$$-----$ (500, 8) (197, 8) (65, 8) (65, 8)
#prepare train set
<class 'list'>
<class 'list'>
<class 'list'>
@$$$--x---$ (2000, 64, 64, 3) (3500, 64, 64, 3) (2000, 64, 64, 3) @@ (500, 8) (197, 8) (65, 8) (65, 8)
@@@@@@@ 3500 2000 2000
# start model selection
INFO:tensorflow:Reloading Oracle from existing project .\image_classifier\oracle.json
INFO:tensorflow:Reloading Tuner from .\image_classifier\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/25
32/32 [==============================] - 18s 552ms/step - loss: 1.2163 - accuracy: 0.6370
Epoch 2/25
32/32 [==============================] - 18s 557ms/step - loss: 1.0067 - accuracy: 0.6870
Epoch 3/25
32/32 [==============================] - 19s 584ms/step - loss: 0.9293 - accuracy: 0.6890
Epoch 4/25
32/32 [==============================] - 21s 664ms/step - loss: 0.9351 - accuracy: 0.6940
Epoch 5/25
32

INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


INFO:tensorflow:Assets written to: .\image_classifier\best_model\assets


#Evaluate the classifier on test data 
32/32 [==============================] - 5s 157ms/step - loss: 0.8546 - accuracy: 0.6760
Accuracy =  67.59999990463257 %
# get the final best performing model 
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 cast_to_float32 (CastToFloa  (None, 64, 64, 3)        0         
 t32)                                                            
                                                                 
 normalization (Normalizatio  (None, 64, 64, 3)        7         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                            

INFO:tensorflow:Assets written to: ../pybin/models/activemodeltest\assets


INFO:tensorflow:Assets written to: ../pybin/models/activemodeltest\assets


In [8]:

import tensorflow as tf
from tensorflow import keras
#train model
#X_train_balanced,Y_cat_train_balanced = prepare_set(df_train_balanced)
#X_test,Y_cat_test = prepare_set(df_test)
#X_validate,Y_cat_validate = prepare_set(df_validate)
model.save('../pybin/models/activemodel_64')

model = keras.models.load_model('../pybin/models/activemodeltest')

model.fit(X_train_balanced,Y_cat_train_balanced,epochs=10)
res=model.evaluate(X_test,Y_cat_test)
print(res)
print('#Save the model')
'#Save the model'
model.save('../pybin/models/activemodel_64_trained')

##

INFO:tensorflow:Assets written to: ../pybin/models/activemodel_64\assets


INFO:tensorflow:Assets written to: ../pybin/models/activemodel_64\assets


Epoch 1/10
110/110 [==============================] - 75s 675ms/step - loss: 1.6012 - accuracy: 0.3629
Epoch 2/10
110/110 [==============================] - 74s 675ms/step - loss: 1.3664 - accuracy: 0.4549
Epoch 3/10
110/110 [==============================] - 74s 675ms/step - loss: 1.2685 - accuracy: 0.5069
Epoch 4/10
110/110 [==============================] - 78s 711ms/step - loss: 1.1739 - accuracy: 0.5431
Epoch 5/10
110/110 [==============================] - 73s 666ms/step - loss: 1.1348 - accuracy: 0.5663
Epoch 6/10
110/110 [==============================] - 74s 670ms/step - loss: 1.0351 - accuracy: 0.6029
Epoch 7/10
110/110 [==============================] - 74s 671ms/step - loss: 1.0293 - accuracy: 0.6186
Epoch 8/10
110/110 [==============================] - 73s 664ms/step - loss: 0.9661 - accuracy: 0.6386
Epoch 9/10
110/110 [==============================] - 73s 664ms/step - loss: 0.9219 - accuracy: 0.6517
Epoch 10/10
63/63 [==============================] - 10s 152ms/step - los

INFO:tensorflow:Assets written to: ../pybin/models/activemodel_64_trained\assets


INFO:tensorflow:Assets written to: ../pybin/models/activemodel_64_trained\assets
